<a href="https://colab.research.google.com/github/jboca-andes/Recursos/blob/main/Tarea_calificada_Perfilamiento_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop2.7.tgz
!tar xf spark-3.2.0-bin-hadoop2.7.tgz

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop2.7"

In [3]:
!pip install -q findspark
!pip install -q pyspark
!pip install -q spark-df-profiling

     |████████████████████████████████| 281.3 MB 53 kB/s 
     |████████████████████████████████| 198 kB 68.5 MB/s 
     |████████████████████████████████| 91 kB 6.4 MB/s 


In [4]:
import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [6]:
aeropuertos = 'aeropuertos.csv'
vuelos = 'vuelos.csv'
from pyspark import SparkFiles
spark.sparkContext.addFile(f'https://raw.githubusercontent.com/MIAD-Modelo-Datos/Recursos/main/Infraestructura%20visible/Etapa%201/{aeropuertos}')
spark.sparkContext.addFile(f'https://raw.githubusercontent.com/MIAD-Modelo-Datos/Recursos/main/Infraestructura%20visible/Etapa%201/{vuelos}')

In [7]:
df_aeropuertos = spark.read.csv('file://'+SparkFiles.get(aeropuertos), header=True, inferSchema=True)
df_vuelos = spark.read.csv('file://'+SparkFiles.get(vuelos), header=True, inferSchema=True)

df_aeropuertos.createOrReplaceTempView("aeropuertos")
df_vuelos.createOrReplaceTempView("vuelos")

In [8]:
df_aeropuertos.printSchema()
df_vuelos.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- sigla: string (nullable = true)
 |-- iata: string (nullable = true)
 |-- nombre: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- latitud: double (nullable = true)
 |-- longitud: double (nullable = true)
 |-- propietario: string (nullable = true)
 |-- explotador: string (nullable = true)
 |-- longitud_pista: double (nullable = true)
 |-- ancho_pista: double (nullable = true)
 |-- pbmo: string (nullable = true)
 |-- elevacion: double (nullable = true)
 |-- resolucion: string (nullable = true)
 |-- fecha_construccion: string (nullable = true)
 |-- fecha_vigencia: string (nullable = true)
 |-- clase: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- numero_vuelos_origen: string (nullable = true)
 |-- gcd_departamento: integer (nullable = true)
 |-- gcd_municipio: integer (nullable = true)

root
 |-- ano: integer (nullable 

In [9]:
spark.sql("select min(cast(UNIX_TIMESTAMP(fecha, 'MM/dd/yyyy') as timestamp)) desde, max(cast(UNIX_TIMESTAMP(fecha, 'MM/dd/yyyy') as timestamp)) hasta from  (select concat(case length(mes) when 1 then '0' else '' end, cast(mes as string),'/','01','/',cast(ano as string)) fecha from vuelos where mes>=0 and ano >=0)").show()

+-------------------+-------------------+
|              desde|              hasta|
+-------------------+-------------------+
|2010-01-01 00:00:00|2013-12-01 00:00:00|
+-------------------+-------------------+



In [10]:
spark.sql('''
    select 
    sigla, iata, nombre, municipio, departamento, 
    categoria, cast(latitud as string) latitud, cast(longitud as string) longitud, 
    propietario, explotador, longitud_pista, ancho_pista, cast(pbmo as double) pbmo, 
    elevacion, resolucion, cast(fecha_construccion as date) fecha_construccion,
    cast(fecha_vigencia as date) fecha_vigencia, clase, tipo, cast(numero_vuelos_origen as int) numero_vuelos_origen,
    cast(gcd_departamento as string) gcd_departamento, cast(gcd_municipio as string) gcd_municipio
    from aeropuertos
''').createOrReplaceTempView("aeropuertos_ok")


spark.sql('''
  select cast(ano as string) ano, cast(mes as string) mes, origen, destino, tipo_equipo, tipo_vuelo, trafico, empresa, 
  cast(vuelos as double) vuelos, cast(sillas as double) sillas, carga_ofrecida, cast(pasajeros as double) pasajeros, carga_bordo
  from vuelos
''').createOrReplaceTempView('vuelos_ok')

In [36]:
spark.sql("select * from vuelos_ok").summary('count', 'mean', 'stddev', 'min', 'max').show()

+-------+------------------+-----------------+------+-------+------------------+----------+-------+--------------------+------+------+------------------+---------+------------------+
|summary|               ano|              mes|origen|destino|       tipo_equipo|tipo_vuelo|trafico|             empresa|vuelos|sillas|    carga_ofrecida|pasajeros|       carga_bordo|
+-------+------------------+-----------------+------+-------+------------------+----------+-------+--------------------+------+------+------------------+---------+------------------+
|  count|             67599|            67599| 67599|  67599|             67599|     67599|  67599|               67599| 67599| 67599|             67599|    67599|             67599|
|   mean|2011.5928637997604|6.674359088152191|  null|   null| 425.0409356725146|      null|    NaN|                null|   NaN|   NaN| 46758.48664891492|      NaN| 5325.677899968939|
| stddev|1.1162151375421105|3.442955191548653|  null|   null|182.93161951904202|     

In [35]:
spark.sql("select * from aeropuertos_ok").summary('count', 'mean', 'stddev', 'min', 'max').show()

+-------+-----+----+---------+---------+------------+---------+------------------+------------------+-----------+----------+-----------------+------------------+-----+------------------+----------+-----+---------+--------------------+-----------------+------------------+
|summary|sigla|iata|   nombre|municipio|departamento|categoria|           latitud|          longitud|propietario|explotador|   longitud_pista|       ancho_pista| pbmo|         elevacion|resolucion|clase|     tipo|numero_vuelos_origen| gcd_departamento|     gcd_municipio|
+-------+-----+----+---------+---------+------------+---------+------------------+------------------+-----------+----------+-----------------+------------------+-----+------------------+----------+-----+---------+--------------------+-----------------+------------------+
|  count|  292| 292|      292|      292|         292|      292|               292|               292|        292|       292|              292|               292|  292|               29

In [12]:
spark.sql('select count(sigla) from aeropuertos_ok').show()

+------------+
|count(sigla)|
+------------+
|         292|
+------------+



## Pregunta:
Visualizar el número de pasajeros transportados en un rango de fechas de aeropuertos Internacionales a Colombia

**Respuesta**: *No es posible responder a la pregunta, los conjuntos de datos no contienen información de vuelos entrantes al territorio Colombiano, sin embargo, es posible conocer el detalle de los pasajeros transportados desde y hacia aeropuertos Internacionales dentro del territorio nacional, adicionalmente, la granularidad disponible se encuentra a nivel de Año-Mes, lo que impide que se pueda realizar un análisis por fecha, se validó con la Aerocivil y dicha información no es de caracter publico.* 
[Datos abiertos Aerocivil](https://www.aerocivil.gov.co/atencion/transparencia/datos-abiertos)

In [13]:
# Pasajeros transportados desde aeropuertos internacionales en el territorio nacional por año y mes.
spark.sql("select ano, mes, sum(pasajeros) pasajeros_transportados from vuelos v inner join aeropuertos a on v.origen = a.sigla where a.categoria = 'Internacional' and v.pasajeros >= 0 group by v.ano, v.mes").show(10)

+----+---+-----------------------+
| ano|mes|pasajeros_transportados|
+----+---+-----------------------+
|2012| 10|              2107605.0|
|2010| 12|              2023021.0|
|2010|  7|              1842447.0|
|2013|  2|              1648071.0|
|2012|  8|              1906273.0|
|2012|  4|              1702058.0|
|2012| 12|              2358191.0|
|2013|  9|              1904437.0|
|2013| 10|              2303307.0|
|2013| 12|              1981243.0|
+----+---+-----------------------+
only showing top 10 rows



In [14]:
# Pasajeros transportados hacia aeropuertos internacionales en el territorio nacional por año y mes.
spark.sql("select ano, mes, sum(pasajeros) pasajeros_transportados from vuelos v inner join aeropuertos a on v.destino = a.sigla where a.categoria = 'Internacional' and v.pasajeros >= 0 group by v.ano, v.mes").show(10)

+----+---+-----------------------+
| ano|mes|pasajeros_transportados|
+----+---+-----------------------+
|2012| 10|              2027717.0|
|2010|  7|              1854910.0|
|2010| 12|              1968860.0|
|2012|  8|              1795754.0|
|2013|  2|              1751075.0|
|2012|  4|              1629414.0|
|2012| 12|              2224497.0|
|2013|  9|              1981828.0|
|2013| 10|              2268371.0|
|2013| 12|              2004059.0|
+----+---+-----------------------+
only showing top 10 rows



## Pregunta:
***Visualizar la carga transportada en un rango de fechas de aeropuertos Internacionales a Colombia***

**Respuesta**: *No es posible responder a la pregunta, los conjuntos de datos no contienen información de vuelos entrantes al territorio Colombiano, sin embargo, es posible conocer el detalle de la carga transportada desde y hacia aeropuertos Internacionales dentro del territorio nacional, adicionalmente, la granularidad disponible se encuentra a nivel de Año-Mes, lo que impide que se pueda realizar un análisis por fecha, se validó con la Aerocivil y dicha información no es de caracter publico.* 
[Datos abiertos Aerocivil](https://www.aerocivil.gov.co/atencion/transparencia/datos-abiertos)

In [15]:
# Carga transportada desde aeropuertos internacionales en el territorio nacional por año y mes.
spark.sql("select ano, mes, sum(carga_bordo) carga from vuelos v inner join aeropuertos a on v.origen = a.sigla where a.categoria = 'Internacional' and v.carga_bordo >= 0 group by v.ano, v.mes").show(10)

+----+---+--------------------+
| ano|mes|               carga|
+----+---+--------------------+
|2012| 10|1.0652734819999995E7|
|2010|  7|          8999624.47|
|2010| 12|       1.630879511E7|
|2013|  2|   7831548.309999999|
|2012|  8|   8871730.169999998|
|2012|  4|1.0231764119999997E7|
|2012| 12|1.3609474729999997E7|
|2013|  9|1.5701067399999999E7|
|2013| 10|1.5676513499999998E7|
|2013| 12|       1.647605157E7|
+----+---+--------------------+
only showing top 10 rows



In [16]:
# Carga transportada hacia aeropuertos internacionales en el territorio nacional por año y mes.
spark.sql("select ano, mes, sum(carga_bordo) carga from vuelos v inner join aeropuertos a on v.destino = a.sigla where a.categoria = 'Internacional' and v.carga_bordo >= 0 group by v.ano, v.mes").show(10)

+----+---+--------------------+
| ano|mes|               carga|
+----+---+--------------------+
|2012| 10|   9094384.079999998|
|2010|  7|          7420303.08|
|2010| 12|1.3420641440000001E7|
|2012|  8|   7153734.279999999|
|2013|  2|   7196776.220000001|
|2012|  4|   9038773.819999998|
|2012| 12|1.1701853099999998E7|
|2013|  9|1.4805132079999998E7|
|2013| 10|1.2822389030000001E7|
|2013| 12|       1.500696409E7|
+----+---+--------------------+
only showing top 10 rows



## Pregunta:
***Visualizar la carga desaprovechada en un rango de fechas de aeropuertos Internacionales a Colombia***

**Respuesta**: *No es posible responder a la pregunta, los conjuntos de datos no contienen información de vuelos entrantes al territorio Colombiano, sin embargo, es posible conocer el detalle de la carga desaprovechada desde y hacia aeropuertos Internacionales dentro del territorio nacional, adicionalmente, la granularidad disponible se encuentra a nivel de Año-Mes, lo que impide que se pueda realizar un análisis por fecha, se validó con la Aerocivil y dicha información no es de caracter publico.* 
[Datos abiertos Aerocivil](https://www.aerocivil.gov.co/atencion/transparencia/datos-abiertos)

In [17]:
# Carga desaprovechada hacia aeropuertos internacionales en el territorio nacional por año y mes.
spark.sql("select ano, mes, sum(carga_ofrecida - carga_bordo) carga_desaprovechada from vuelos v inner join aeropuertos a on v.destino = a.sigla where a.categoria = 'Internacional' and v.carga_ofrecida >= 0 group by v.ano, v.mes").show(10)

+----+---+--------------------+
| ano|mes|carga_desaprovechada|
+----+---+--------------------+
|2012| 10|1.2467720043999998E8|
|2010|  7|       5.361381257E7|
|2010| 12|      1.3593209244E8|
|2012|  8| 9.274949392999999E7|
|2013|  2| 8.566440394000001E7|
|2012|  4| 8.837291497999999E7|
|2012| 12|      1.1447950613E8|
|2013|  9|       8.703813912E7|
|2013| 10|1.1072354300999999E8|
|2013| 12|1.1420259094000003E8|
+----+---+--------------------+
only showing top 10 rows



In [18]:
# Carga desaprovechada desde aeropuertos internacionales en el territorio nacional por año y mes.
spark.sql("select ano, mes, sum(carga_ofrecida - carga_bordo) carga_desaprovechada from vuelos v inner join aeropuertos a on v.origen = a.sigla where a.categoria = 'Internacional' and v.carga_ofrecida >= 0 group by v.ano, v.mes").show(10)

+----+---+--------------------+
| ano|mes|carga_desaprovechada|
+----+---+--------------------+
|2012| 10|      1.2357486432E8|
|2010|  7|       5.227463391E7|
|2010| 12|1.1299611522999999E8|
|2013|  2|       7.952276012E7|
|2012|  8|       9.104578136E7|
|2012|  4| 9.270638188999996E7|
|2012| 12|       1.111906144E8|
|2013|  9|       9.182402302E7|
|2013| 10|1.0880657265999998E8|
|2013| 12|1.0423506235000001E8|
+----+---+--------------------+
only showing top 10 rows



## Pregunta:
***Visualizar el número de puestos desaprovechados en un rango de fechas de aeropuertos Internacionales a Colombia***

**Respuesta**: *No es posible responder a la pregunta, los conjuntos de datos no contienen información de vuelos entrantes al territorio Colombiano, sin embargo, es posible conocer el detalle de los puestos desaprovechados desde y hacia aeropuertos Internacionales dentro del territorio nacional, adicionalmente, la granularidad disponible se encuentra a nivel de Año-Mes, lo que impide que se pueda realizar un análisis por fecha, se validó con la Aerocivil y dicha información no es de caracter publico.* 
[Datos abiertos Aerocivil](https://www.aerocivil.gov.co/atencion/transparencia/datos-abiertos)

In [19]:
# Puestos desaprovechados desde aeropuertos internacionales en el territorio nacional por año y mes.
spark.sql("select ano, mes, sum(sillas - pasajeros) puestos_desaprovechados from vuelos v inner join aeropuertos a on v.origen = a.sigla where a.categoria = 'Internacional' and v.sillas >= 0 and v.pasajeros >= 0 group by v.ano, v.mes").show(10)

+----+---+-----------------------+
| ano|mes|puestos_desaprovechados|
+----+---+-----------------------+
|2012| 10|               458819.0|
|2010| 12|               419584.0|
|2010|  7|               328342.0|
|2013|  2|               357415.0|
|2012|  8|               365439.0|
|2012|  4|               564234.0|
|2012| 12|               425236.0|
|2013|  9|               366431.0|
|2013| 10|               450551.0|
|2013| 12|               351656.0|
+----+---+-----------------------+
only showing top 10 rows



In [20]:
# Puestos desaprovechados hacia aeropuertos internacionales en el territorio nacional por año y mes.
spark.sql("select ano, mes, sum(sillas - pasajeros) puestos_desaprovechados from vuelos v inner join aeropuertos a on v.destino = a.sigla where a.categoria = 'Internacional' and v.sillas >= 0 and v.pasajeros >= 0 group by v.ano, v.mes").show(10)

+----+---+-----------------------+
| ano|mes|puestos_desaprovechados|
+----+---+-----------------------+
|2012| 10|               441357.0|
|2010|  7|               301175.0|
|2010| 12|               562669.0|
|2012|  8|               335900.0|
|2013|  2|               355057.0|
|2012|  4|               537719.0|
|2012| 12|               514333.0|
|2013|  9|               360373.0|
|2013| 10|               432513.0|
|2013| 12|               452507.0|
+----+---+-----------------------+
only showing top 10 rows



In [21]:
spark.sql("select distinct categoria from aeropuertos").show()

+-------------+
|    categoria|
+-------------+
|     Nacional|
|Internacional|
|    Aeródromo|
|     Regional|
+-------------+



In [22]:
spark.sql("select * from aeropuertos where iata<>sigla limit 10").show()

+---+-----+----+----------------+--------------------+------------+---------+-------+--------+--------------------+--------------------+--------------+-----------+-------+---------+----------+------------------+--------------+-----+----------+--------------------+----------------+-------------+
|_c0|sigla|iata|          nombre|           municipio|departamento|categoria|latitud|longitud|         propietario|          explotador|longitud_pista|ancho_pista|   pbmo|elevacion|resolucion|fecha_construccion|fecha_vigencia|clase|      tipo|numero_vuelos_origen|gcd_departamento|gcd_municipio|
+---+-----+----+----------------+--------------------+------------+---------+-------+--------+--------------------+--------------------+--------------+-----------+-------+---------+----------+------------------+--------------+-----+----------+--------------------+----------------+-------------+
|134|  9DG| nan|    EL LAGO - CA|             Piedras|      Tolima|Aeródromo| 4.5575|-74.9003|    BERNEY RODRIGU

In [23]:
spark.sql("select * from vuelos limit 10").show()

+----+---+------+-------+-----------+----------+-------+--------------------+------+------+--------------+---------+-----------+
| ano|mes|origen|destino|tipo_equipo|tipo_vuelo|trafico|             empresa|vuelos|sillas|carga_ofrecida|pasajeros|carga_bordo|
+----+---+------+-------+-----------+----------+-------+--------------------+------+------+--------------+---------+-----------+
|2012|  1|   BOG|    CUC|       BE20|         T|      N|             RIO SUR|   1.0|   0.0|           0.0|      4.0|      100.0|
|2013|  5|   UIB|    BOG|       DH8D|         R|      N|               AIRES|  30.0|1110.0|       24000.0|    873.0|     4222.0|
|2013| 10|   IBE|    BOG|       DH8D|         R|      N|               AIRES|  98.0|3626.0|       56056.0|   2866.0|    2323.75|
|2012|  4|   FLA|    BOG|       JS32|         T|      N|         SARPA S.A.S|   1.0|   0.0|           0.0|      4.0|        0.0|
|2013|  7|   CUC|    AUC|       C402|         T|      N|AEROCHARTER ANDIN...|   1.0|   0.0|      

In [24]:
df_aeropuertos.selectExpr("longitud_pista", "ancho_pista", "cast(pbmo as int) pbmo", "elevacion", "cast(numero_vuelos_origen as int) numero_vuelos_origen").toPandas().describe()

,longitud_pista,ancho_pista,pbmo,elevacion,numero_vuelos_origen
count,292.000000,292.000000,237.000000,292.000000,2.210000e+02
mean,962.445205,17.633562,5683.341772,1073.308219,1.855764e+05
std,540.019110,10.356164,9669.194412,1580.858623,3.121902e+05
min,200.000000,10.000000,400.000000,0.000000,1.000000e+00
25%,700.000000,10.000000,2000.000000,357.000000,5.100000e+01
50%,800.000000,15.000000,2000.000000,597.000000,5.850000e+03
75%,1200.000000,20.000000,4500.000000,870.000000,2.403310e+05
max,3800.000000,75.000000,63196.000000,9740.000000,1.804153e+06


In [25]:
df_vuelos.selectExpr("carga_ofrecida", "carga_bordo", "cast(sillas as int) sillas", "cast(vuelos as int) vuelos", "cast(pasajeros as int) pasajeros").toPandas().describe()

,carga_ofrecida,carga_bordo,sillas,vuelos,pasajeros
count,6.759900e+04,6.759900e+04,63582.000000,66017.000000,64116.000000
mean,4.675849e+04,5.325678e+03,1133.122535,14.263735,911.728430
std,2.306673e+05,2.341590e+04,4468.301902,40.198515,3613.766211
min,0.000000e+00,0.000000e+00,0.000000,1.000000,0.000000
25%,0.000000e+00,0.000000e+00,0.000000,1.000000,2.000000
50%,0.000000e+00,1.100000e+02,0.000000,2.000000,8.000000
75%,9.000000e+03,1.338000e+03,240.000000,9.000000,184.000000
max,7.361385e+06,1.062015e+06,79925.000000,2004.000000,65195.000000
